In [22]:
# !pip install opencv-python
# !pip install scikit-image
# !pip install astropy

import cv2
import numpy as np
import os
from astropy.io import fits
from skimage.measure import label, regionprops

# Define paths
input_folder = "C:\\Users\\incar\\Desktop\\Astrofotografia\\MyAstro\\Seestar\\NGC 6946-sub"
output_folder = "C:\\Users\\incar\\Desktop\\Astrofotografia\\MyAstro\\Seestar\\NGC 6946-sub\\low_quality_images"
os.makedirs(output_folder, exist_ok=True)

In [21]:
# Quality thresholds
blur_threshold = 80.0  # Adjusted threshold
roundness_threshold = 0.6  # Minimum acceptable star roundness (1.0 = perfect circle)
filter_out_noise = 5

# Weights for quality score
blur_weight = 0.4
roundness_weight = 0.6
quality_threshold = 50.0  # Overall quality score threshold

def calculate_star_roundness(image):
    """Detect stars and calculate their roundness."""
    _, binary = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY)  # Binarize image
    labeled = label(binary)  # Label connected components
    properties = regionprops(labeled)
    
    roundness_scores = []
    for prop in properties:
        if prop.area > filter_out_noise:  # Filter out noise (small areas)
            minor_axis = prop.minor_axis_length
            major_axis = prop.major_axis_length
            if major_axis > 0:  # Avoid division by zero
                roundness = minor_axis / major_axis
                roundness_scores.append(roundness)
    
    return roundness_scores

# Loop through `.fit` files
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".fit"):
        filepath = os.path.join(input_folder, filename)
        
        # Load FITS file
        with fits.open(filepath) as hdul:
            image_data = hdul[0].data  # Assuming the image is in the primary HDU
            
        # Normalize data for OpenCV
        image_data = np.nan_to_num(image_data)  # Handle NaNs
        image_data = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Check for blurriness
        blur_variance = cv2.Laplacian(image_data, cv2.CV_64F).var()
        
        # Calculate star roundness
        roundness_scores = calculate_star_roundness(image_data)
        avg_roundness = np.mean(roundness_scores) if roundness_scores else 0
        
        # Calculate overall quality score
        quality_score = (blur_weight * blur_variance) + (roundness_weight * avg_roundness * 100)  # Scale roundness
        
        # Apply filters
        if quality_score < quality_threshold:
            # Move low-quality image to output folder
            os.rename(filepath, os.path.join(output_folder, filename))
            print(f"Moved {filename} (Blur: {blur_variance:.2f}, Roundness: {avg_roundness:.2f}, Quality Score: {quality_score:.2f})")
        else:
            print(f"Kept {filename} (Blur: {blur_variance:.2f}, Roundness: {avg_roundness:.2f}, Quality Score: {quality_score:.2f})")


Kept Light_NGC 6946_10.0s_IRCUT_20240804-215544.fit (Blur: 7.12, Roundness: 0.98, Quality Score: 61.69)
Moved Light_NGC 6946_10.0s_IRCUT_20240804-215555.fit (Blur: 6.62, Roundness: 0.64, Quality Score: 41.16)
Moved Light_NGC 6946_10.0s_IRCUT_20240804-215606.fit (Blur: 6.22, Roundness: 0.77, Quality Score: 48.43)
Moved Light_NGC 6946_10.0s_IRCUT_20240804-215617.fit (Blur: 6.67, Roundness: 0.74, Quality Score: 46.84)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215628.fit (Blur: 6.55, Roundness: 0.87, Quality Score: 54.73)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215648.fit (Blur: 6.24, Roundness: 0.90, Quality Score: 56.47)
Moved Light_NGC 6946_10.0s_IRCUT_20240804-215700.fit (Blur: 6.57, Roundness: 0.76, Quality Score: 48.12)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215711.fit (Blur: 6.83, Roundness: 0.90, Quality Score: 56.72)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215722.fit (Blur: 6.78, Roundness: 0.83, Quality Score: 52.23)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215733.fit (Blur: 6

In [ ]:


# Quality thresholds
blur_threshold = 80.0  # Adjusted threshold
roundness_threshold = 0.6  # Minimum acceptable star roundness (1.0 = perfect circle)
filter_out_noise = 5

# Weights for quality score
blur_weight = 0.4
roundness_weight = 0.6
quality_threshold = 50.0  # Overall quality score threshold

# Parameters for plane trail detection
motion_blur_threshold = 100  # Threshold for detecting motion blur
min_aspect_ratio = 3.0  # Minimum aspect ratio to classify as a plane trail

def calculate_star_roundness(image):
    """Detect stars and calculate their roundness."""
    _, binary = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY)  # Binarize image
    labeled = label(binary)  # Label connected components
    properties = regionprops(labeled)
    
    roundness_scores = []
    for prop in properties:
        if prop.area > filter_out_noise:  # Filter out noise (small areas)
            minor_axis = prop.minor_axis_length
            major_axis = prop.major_axis_length
            if major_axis > 0:  # Avoid division by zero
                roundness = minor_axis / major_axis
                roundness_scores.append(roundness)
    
    return roundness_scores

def detect_plane_trail(image):
    """Detect plane trails by checking for motion blur or elongated structures."""
    # Since the image is already grayscale, no need to convert it again
    gray = image
    
    # Edge detection (Canny)
    edges = cv2.Canny(gray, 50, 150)
    
    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        # Calculate the bounding box of each contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Calculate aspect ratio (width/height)
        aspect_ratio = float(w) / h if h != 0 else 0
        
        # Check if aspect ratio is large enough to indicate a plane trail
        if aspect_ratio > min_aspect_ratio:
            return True  # Plane trail detected
    
    return False  # No plane trail detected


# Loop through `.fit` files
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".fit"):
        filepath = os.path.join(input_folder, filename)
        
        # Load FITS file
        with fits.open(filepath) as hdul:
            image_data = hdul[0].data  # Assuming the image is in the primary HDU
            
        # Normalize data for OpenCV
        image_data = np.nan_to_num(image_data)  # Handle NaNs
        image_data = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Check for blurriness
        blur_variance = cv2.Laplacian(image_data, cv2.CV_64F).var()
        
        # Calculate star roundness
        roundness_scores = calculate_star_roundness(image_data)
        avg_roundness = np.mean(roundness_scores) if roundness_scores else 0
        
        # Detect plane trails
        if detect_plane_trail(image_data):
            print(f"Excluded {filename} due to plane trail detection")
            continue
        
        # Calculate overall quality score
        quality_score = (blur_weight * blur_variance) + (roundness_weight * avg_roundness * 100)  # Scale roundness
        
        # Apply filters
        if quality_score < quality_threshold:
            # Move low-quality image to output folder
            os.rename(filepath, os.path.join(output_folder, filename))
            print(f"Moved {filename} (Blur: {blur_variance:.2f}, Roundness: {avg_roundness:.2f}, Quality Score: {quality_score:.2f})")
        else:
            print(f"Kept {filename} (Blur: {blur_variance:.2f}, Roundness: {avg_roundness:.2f}, Quality Score: {quality_score:.2f})")


Kept Light_NGC 6946_10.0s_IRCUT_20240804-215544.fit (Blur: 7.12, Roundness: 0.98, Quality Score: 61.69)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215628.fit (Blur: 6.55, Roundness: 0.87, Quality Score: 54.73)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215648.fit (Blur: 6.24, Roundness: 0.90, Quality Score: 56.47)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215711.fit (Blur: 6.83, Roundness: 0.90, Quality Score: 56.72)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215722.fit (Blur: 6.78, Roundness: 0.83, Quality Score: 52.23)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215733.fit (Blur: 6.15, Roundness: 0.83, Quality Score: 52.24)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215817.fit (Blur: 6.02, Roundness: 0.83, Quality Score: 52.38)
Excluded Light_NGC 6946_10.0s_IRCUT_20240804-215840.fit due to plane trail detection
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215902.fit (Blur: 6.06, Roundness: 0.80, Quality Score: 50.18)
Kept Light_NGC 6946_10.0s_IRCUT_20240804-215913.fit (Blur: 5.77, Roundness: 0.86, Q